In [ ]:
import torch
import torch.nn.functional as F

# Two 1D points
x1 = torch.tensor([1.0, 2.0])
x2 = torch.tensor([1.1, 2.1])
x3 = torch.tensor([15.0, 5.0])

# cosine similarity
def similarity(a, b):
    return F.cosine_similarity(a, b, dim=0)

# Compute similarities
sim_positive = similarity(x1, x2)  # should be high
sim_negative = similarity(x1, x3)  # should be low

print(f"Similarity of positive pair (x1,x2): {sim_positive.item():.2f}")
print(f"Similarity of negative pair (x1,x3): {sim_negative.item():.2f}")


Similarity of positive pair (x1,x2): 1.00
Similarity of negative pair (x1,x3): 0.71


In [ ]:
import torch
import torch.nn.functional as F
import torch.optim as optim


# Initialize "embeddings" from scratch
# Let's say we have 3 points in 2D space (random init)
z1 = torch.randn(2, requires_grad=True)
z2 = torch.randn(2, requires_grad=True)
z3 = torch.randn(2, requires_grad=True)  # negative

# Cosine similarity function
def similarity(a, b):
    return F.cosine_similarity(a, b, dim=0)

# Learning rate
lr = 0.1

optimizer = optim.SGD([z1, z2, z3], lr=0.1)

# Training loop
for step in range(100):

    optimizer.zero_grad()

    # Cosine similarities
    sim_pos = similarity(z1, z2)
    sim_neg = similarity(z1, z3)

    # Contrastive loss: maximize positive, minimize negative
    loss = -(sim_pos - sim_neg)

    # Compute gradients
    loss.backward()

    optimizer.step()

    if step % 10 == 0:
        print(f"Step {step}: loss={loss.item():.3f}, sim_pos={sim_pos.item():.3f}, sim_neg={sim_neg.item():.3f}")

# Final embeddings
print("\nFinal embeddings:")
print("z1:", z1)
print("z2:", z2)
print("z3:", z3)


Step 0: loss=-0.011, sim_pos=-0.989, sim_neg=-1.000
Step 10: loss=-0.074, sim_pos=-0.913, sim_neg=-0.988
Step 20: loss=-0.378, sim_pos=-0.544, sim_neg=-0.923
Step 30: loss=-1.040, sim_pos=0.239, sim_neg=-0.800
Step 40: loss=-1.526, sim_pos=0.720, sim_neg=-0.806
Step 50: loss=-1.771, sim_pos=0.882, sim_neg=-0.889
Step 60: loss=-1.892, sim_pos=0.946, sim_neg=-0.946
Step 70: loss=-1.950, sim_pos=0.975, sim_neg=-0.975
Step 80: loss=-1.977, sim_pos=0.989, sim_neg=-0.988
Step 90: loss=-1.989, sim_pos=0.995, sim_neg=-0.995

Final embeddings:
z1: tensor([-0.1406,  1.0163], requires_grad=True)
z2: tensor([-0.3305,  1.5748], requires_grad=True)
z3: tensor([ 0.1098, -1.6142], requires_grad=True)


In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


x1 = torch.tensor([1.0, 2.0])
x2 = torch.tensor([1.1, 2.1])  # positive
x3 = torch.tensor([15.0, 5.0]) # negative

class Encoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(2, 4)  # hidden layer 2 → 4
        self.fc2 = nn.Linear(4, 2)  # output layer 4 → 2
    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Cosine similarity function
def similarity(a, b):
    return F.cosine_similarity(a, b, dim=0)

# Learning rate
lr = 0.1

encoder = Encoder()
print(encoder.parameters())
optimizer = optim.SGD(encoder.parameters(), lr=0.1)

# Training loop
for step in range(100):
    z1 = encoder(x1)
    z2 = encoder(x2)
    z3 = encoder(x3)

    optimizer.zero_grad()

    # Cosine similarities
    sim_pos = similarity(z1, z2)
    sim_neg = similarity(z1, z3)

    # Contrastive loss: maximize positive, minimize negative
    loss = -(sim_pos - sim_neg)

    # Compute gradients
    loss.backward()

    optimizer.step()

    if step % 10 == 0:
        print(f"Step {step}: loss={loss.item():.3f}, sim_pos={sim_pos.item():.3f}, sim_neg={sim_neg.item():.3f}")

# Final embeddings
z1 = encoder(x1)
z2 = encoder(x2)
z3 = encoder(x3)
print("\nFinal embeddings:")
print("z1:", z1)
print("z2:", z2)
print("z3:", z3)


<generator object Module.parameters at 0x7cec58394d60>
Step 0: loss=-0.131, sim_pos=0.999, sim_neg=0.868
Step 10: loss=-2.000, sim_pos=1.000, sim_neg=-1.000
Step 20: loss=-2.000, sim_pos=1.000, sim_neg=-1.000
Step 30: loss=-2.000, sim_pos=1.000, sim_neg=-1.000
Step 40: loss=-2.000, sim_pos=1.000, sim_neg=-1.000
Step 50: loss=-2.000, sim_pos=1.000, sim_neg=-1.000
Step 60: loss=-2.000, sim_pos=1.000, sim_neg=-1.000
Step 70: loss=-2.000, sim_pos=1.000, sim_neg=-1.000
Step 80: loss=-2.000, sim_pos=1.000, sim_neg=-1.000
Step 90: loss=-2.000, sim_pos=1.000, sim_neg=-1.000

Final embeddings:
z1: tensor([-0.2879,  0.6153], grad_fn=<ViewBackward0>)
z2: tensor([-0.2802,  0.6018], grad_fn=<ViewBackward0>)
z3: tensor([ 1.7130, -3.6618], grad_fn=<ViewBackward0>)


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Dataset + simple transform ---
transform = transforms.Compose([
    transforms.ToTensor(),
])

dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
loader = DataLoader(dataset, batch_size=4, shuffle=True)  # small batch

# CNN Encoder ---
class CNNEncoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)  # 28x28 -> 28x28
        self.pool = nn.MaxPool2d(2, 2)  # 28x28 -> 14x14
        self.conv2 = nn.Conv2d(16, 32, 3, 1, 1)  # 14x14 -> 14x14
        self.fc1 = nn.Linear(32*14*14, 128)  # flatten -> 128-dim embedding

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = x.view(x.size(0), -1)  # flatten
        x = self.fc1(x)
        x = F.normalize(x, dim=1)  # normalize embedding
        return x

encoder = CNNEncoder()
optimizer = optim.SGD(encoder.parameters(), lr=0.01)

#  Training loop ---
for step, (images, labels) in enumerate(loader):
    if step > 50:  # few batches for demo
        break

    optimizer.zero_grad()

    # Take first 3 images as anchor, positive, negative
    img_anchor = images[0].unsqueeze(0)   # add batch dim
    img_positive = images[1].unsqueeze(0)
    img_negative = images[2].unsqueeze(0)

    z_anchor = encoder(img_anchor)
    z_pos = encoder(img_positive)
    z_neg = encoder(img_negative)

    sim_pos = F.cosine_similarity(z_anchor, z_pos)
    sim_neg = F.cosine_similarity(z_anchor, z_neg)

    loss = -(sim_pos - sim_neg)
    loss.backward()
    optimizer.step()

    print(f"Step {step}: loss={loss.item():.3f}, sim_pos={sim_pos.item():.3f}, sim_neg={sim_neg.item():.3f}")

# Print learned parameters ---
print("\nLearned parameters:")
for name, param in encoder.named_parameters():
    print(name, param.shape)


Step 0: loss=-0.125, sim_pos=0.662, sim_neg=0.536
Step 1: loss=-0.119, sim_pos=0.477, sim_neg=0.358
Step 2: loss=0.156, sim_pos=0.660, sim_neg=0.816
Step 3: loss=0.038, sim_pos=0.894, sim_neg=0.932
Step 4: loss=0.217, sim_pos=0.729, sim_neg=0.947
Step 5: loss=0.002, sim_pos=0.941, sim_neg=0.943
Step 6: loss=0.015, sim_pos=0.934, sim_neg=0.949
Step 7: loss=-0.016, sim_pos=0.930, sim_neg=0.914
Step 8: loss=0.009, sim_pos=0.873, sim_neg=0.882
Step 9: loss=0.047, sim_pos=0.883, sim_neg=0.930
Step 10: loss=0.027, sim_pos=0.896, sim_neg=0.923
Step 11: loss=-0.040, sim_pos=0.976, sim_neg=0.936
Step 12: loss=-0.022, sim_pos=0.954, sim_neg=0.932
Step 13: loss=0.033, sim_pos=0.943, sim_neg=0.975
Step 14: loss=-0.024, sim_pos=0.962, sim_neg=0.937
Step 15: loss=0.021, sim_pos=0.878, sim_neg=0.900
Step 16: loss=0.012, sim_pos=0.954, sim_neg=0.966
Step 17: loss=0.006, sim_pos=0.951, sim_neg=0.957
Step 18: loss=-0.032, sim_pos=0.965, sim_neg=0.933
Step 19: loss=0.031, sim_pos=0.933, sim_neg=0.964
Ste